In [1]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import os
import re
from urllib.parse import urlparse

In [5]:
data_path = 'exp-data/nyt-t1.sqlite'

In [74]:
query = """SELECT sv.site_url, sv.visit_id,
        hr.headers, hr.referrer, hr.url, hr.top_level_url
        FROM http_requests as hr LEFT JOIN site_visits as sv
        ON sv.visit_id = hr.visit_id
        """

In [77]:
conn = sqlite3.connect(data_path)
# http_requests = pd.read_sql_query("select visit_id, headers, referrer, url, top_level_url from http_requests;", conn)
http_requests = pd.read_sql_query(query, conn)

In [78]:
http_requests

,site_url,visit_id,headers,referrer,url,top_level_url
0,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""www.webmd.com""],[""User-Agent"",""Mozil...",,https://www.webmd.com/search/search_results/de...,https://www.webmd.com/search/search_results/de...
1,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""css.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/search/search_results/de...,https://css.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/search/search_results/de...
2,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""css.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/search/search_results/de...,https://css.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/search/search_results/de...
3,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""js.webmd.com""],[""User-Agent"",""Mozill...",https://www.webmd.com/search/search_results/de...,https://js.webmd.com/dtmcms/live/webmd/PageBui...,https://www.webmd.com/search/search_results/de...
4,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""js.webmd.com""],[""User-Agent"",""Mozill...",https://www.webmd.com/search/search_results/de...,https://js.webmd.com/dtmcms/live/webmd/PageBui...,https://www.webmd.com/search/search_results/de...
5,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""img.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/search/search_results/de...,https://img.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/search/search_results/de...
6,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""img.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/search/search_results/de...,https://img.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/search/search_results/de...
7,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""img.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/search/search_results/de...,https://img.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/search/search_results/de...
8,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""img.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/search/search_results/de...,https://img.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/search/search_results/de...
9,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""img.webmd.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/search/search_results/de...,https://img.webmd.com/dtmcms/live/webmd/PageBu...,https://www.webmd.com/search/search_results/de...


In [17]:
cookies = pd.read_sql_query("select visit_id, record_type, change_cause, host, name, value from javascript_cookies;", conn)

In [80]:
def get_host_from_headers(x):
    temp = x.replace('"','').split('],[')
    for t in temp:
            pair = t.replace(']]','').replace('[[','').split(',')
            if "Host" in pair:
                host_value = pair[1]
    return host_value

def get_cookies_from_headers(x):
    ls = x.replace('"','').split('],[')
    for l in ls:
        ta = l.replace(']]','').split(',')
        if "Cookie" in ta:
            return ta[-1]
    return ''
def extract_host_from_url(url_ls):
    return list(map(lambda x: urlparse(x).netloc.split('.')[1] if len(urlparse(x).netloc.split('.')) > 1 else x, url_ls))

def get_site(site_url):
    return site_url.split(".")[1]
def is_third_party(origin, url_host):
    if origin in url_host:
        return False
    return True

In [83]:
http_requests["host"] = list(map(lambda x: get_host_from_headers(x), http_requests.headers.tolist()))
http_requests["cookies"] =  list(map(lambda x: get_cookies_from_headers(x), http_requests.headers.tolist()))
http_requests["origin_site"] = list(map(lambda x: get_site(x), http_requests.site_url))
http_requests["is_third_party"] = list(map(lambda a, b: is_third_party(a, b), http_requests.origin_site, http_requests.host))

In [85]:
http_requests_third_party = http_requests[http_requests.is_third_party == True]

In [86]:
http_requests_third_party

,site_url,visit_id,headers,referrer,url,top_level_url,host,cookies,origin_site,is_third_party
10,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""assets.adobedtm.com""],[""User-Agent"",...",https://www.webmd.com/search/search_results/de...,https://assets.adobedtm.com/2c8c1e17b98c/e6d47...,https://www.webmd.com/search/search_results/de...,assets.adobedtm.com,,webmd,True
22,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""s.flocdn.com""],[""User-Agent"",""Mozill...",https://www.webmd.com/search/search_results/de...,https://s.flocdn.com/@s1/embedded-search/embed...,https://www.webmd.com/search/search_results/de...,s.flocdn.com,,webmd,True
35,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""www.googleadservices.com""],[""User-Ag...",https://www.webmd.com/search/search_results/de...,https://www.googleadservices.com/pagead/conver...,https://www.webmd.com/search/search_results/de...,www.googleadservices.com,,webmd,True
37,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""fonts.googleapis.com""],[""User-Agent""...",https://css.webmd.com/dtmcms/live/webmd/PageBu...,https://fonts.googleapis.com/css?family=Roboto...,https://www.webmd.com/search/search_results/de...,fonts.googleapis.com,,webmd,True
38,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""dyv1bugovvq1g.cloudfront.net""],[""Use...",https://www.webmd.com/search/search_results/de...,https://dyv1bugovvq1g.cloudfront.net/25/www.we...,https://www.webmd.com/search/search_results/de...,dyv1bugovvq1g.cloudfront.net,,webmd,True
39,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""d15kdpgjg3unno.cloudfront.net""],[""Us...",https://www.webmd.com/search/search_results/de...,https://d15kdpgjg3unno.cloudfront.net/oPS.js?c...,https://www.webmd.com/search/search_results/de...,d15kdpgjg3unno.cloudfront.net,,webmd,True
40,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""fonts.gstatic.com""],[""User-Agent"",""M...",https://fonts.googleapis.com/css?family=Roboto...,https://fonts.gstatic.com/s/sourcesanspro/v14/...,https://www.webmd.com/search/search_results/de...,fonts.gstatic.com,,webmd,True
41,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""securepubads.g.doubleclick.net""],[""U...",https://www.webmd.com/search/search_results/de...,https://securepubads.g.doubleclick.net/tag/js/...,https://www.webmd.com/search/search_results/de...,securepubads.g.doubleclick.net,,webmd,True
42,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/search/search_results/de...,https://contextual.media.net/bidexchange.js?ci...,https://www.webmd.com/search/search_results/de...,contextual.media.net,,webmd,True
43,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""z.moatads.com""],[""User-Agent"",""Mozil...",https://www.webmd.com/search/search_results/de...,https://z.moatads.com/webmdheader894912230343/...,https://www.webmd.com/search/search_results/de...,z.moatads.com,,webmd,True


In [89]:
query2 = """SELECT sv.site_url, sv.visit_id,
        jsc.host, jsc.name, jsc.value
        FROM javascript_cookies as jsc LEFT JOIN site_visits as sv
        ON sv.visit_id = jsc.visit_id
        """
cookies = pd.read_sql_query(query2, conn)

In [98]:
# query2 = "select visit_id, record_type, change_cause, host, name, value, time_stamp from javascript_cookies;"
# 
cookies['origin_site'] = list(map(lambda x: get_site(x), cookies.site_url))
cookies['is_third_party'] = list(map(lambda a, b: is_third_party(a, b), cookies.origin_site, cookies.host))

In [101]:
cookies_first_party = cookies[cookies.is_third_party == False]

In [102]:
cookies_first_party

,site_url,visit_id,host,name,value,origin_site,is_third_party
0,https://www.webmd.com/search/search_results/de...,1,.www.webmd.com,__cfduid,de1f86f9e699cddfc6fcaeed623176bae1606506089,webmd,False
1,https://www.webmd.com/search/search_results/de...,1,.js.webmd.com,__cfduid,d8d4ebd1e437ddcb310e31be0d82e1cdb1606506089,webmd,False
2,https://www.webmd.com/search/search_results/de...,1,.img.webmd.com,__cfduid,d5b4e59d78d86d6c5ab29b6c3e948cd431606506089,webmd,False
3,https://www.webmd.com/search/search_results/de...,1,.css.webmd.com,__cfduid,d74794a4d59fa87e1456c8b173f3ff20e1606506089,webmd,False
4,https://www.webmd.com/search/search_results/de...,1,.webmd.com,test,cookie,webmd,False
5,https://www.webmd.com/search/search_results/de...,1,.webmd.com,test,cookie,webmd,False
6,https://www.webmd.com/search/search_results/de...,1,.webmd.com,test,cookie,webmd,False
7,https://www.webmd.com/search/search_results/de...,1,.webmd.com,test,cookie,webmd,False
8,https://www.webmd.com/search/search_results/de...,1,.webmd.com,test,cookie,webmd,False
9,https://www.webmd.com/search/search_results/de...,1,.webmd.com,test,cookie,webmd,False


In [107]:
# cookie_effective_values = [v.replace('|', '\|').replace("-", "\-").replace("[",'\[').replace(']','\]') for v in set(cookies.value) if len(v) > 5]
cookie_effective_values = [re.escape(v) for v in set(cookies_first_party.value) if len(v) > 5]
cookie_effective_all = [re.escape(v) for v in set(cookies.value) if len(v) > 5]

In [108]:
cookie_effective_values

['\\"z\\=1\\&dm\\=webmd\\.com\\&si\\=elgvz2mutr9\\&ss\\=ki0o8vq2\\&sl\\=5\\&tt\\=5qg\\&obo\\=3\\&rl\\=1\\"',
 '\\{\\"ct\\"\\:\\"Columbus\\"\\,\\"c\\"\\:\\"Franklin\\"\\,\\"cc\\"\\:\\"39049\\"\\,\\"st\\"\\:\\"OH\\"\\,\\"sc\\"\\:\\"36\\"\\,\\"z\\"\\:\\"43230\\"\\,\\"lat\\"\\:\\"40\\.04\\"\\,\\"lon\\"\\:\\"\\-82\\.86\\"\\,\\"dma\\"\\:\\"535\\"\\,\\"cntr\\"\\:\\"usa\\"\\,\\"cntrc\\"\\:\\"840\\"\\,\\"tz\\"\\:null\\,\\"ci\\"\\:\\"3\\.16\\.218\\.99\\"\\}',
 '\\-432600572\\%7CMCIDTS\\%7C18594\\%7CMCMID\\%7C45583913191978422602256895716018834768\\%7CMCAAMLH\\-1607110892\\%7C7\\%7CMCAAMB\\-1607110892\\%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y\\%7CvVersion\\%7C4\\.5\\.2',
 'GUUBJm9NaxnvzQyCGj8zEx',
 'GA1\\.2\\.2028454572\\.1606506417',
 '45429554768808693152241494766003155399',
 'lrt1_cached_k8_worker_1_37S_21O_2020\\-11\\-18_17\\:20\\:54_gdbp_false_k8_E_false_UC_false_UD_false_mp_true_active_false_from_MA1_ovr__config_undefined_1606506167202_hvid_true',
 's\\=1606506281646\\&r\\=ht

In [109]:
cookie_values = "|".join(cookie_effective_values)

In [110]:
http_requests_third_party[http_requests_third_party.url.str.contains(cookie_values, regex=True)]

,site_url,visit_id,headers,referrer,url,top_level_url,host,cookies,origin_site,is_third_party
38,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""dyv1bugovvq1g.cloudfront.net""],[""Use...",https://www.webmd.com/search/search_results/de...,https://dyv1bugovvq1g.cloudfront.net/25/www.we...,https://www.webmd.com/search/search_results/de...,dyv1bugovvq1g.cloudfront.net,,webmd,True
51,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""sb.scorecardresearch.com""],[""User-Ag...",https://www.webmd.com/search/search_results/de...,https://sb.scorecardresearch.com/b?c1=2&c2=603...,https://www.webmd.com/search/search_results/de...,sb.scorecardresearch.com,,webmd,True
53,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""s.tagsrvcs.com""],[""User-Agent"",""Mozi...",https://www.webmd.com/search/search_results/de...,https://s.tagsrvcs.com/2/587654/analytics.js?p...,https://www.webmd.com/search/search_results/de...,s.tagsrvcs.com,,webmd,True
69,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""sb.scorecardresearch.com""],[""User-Ag...",https://www.webmd.com/search/search_results/de...,https://sb.scorecardresearch.com/b2?c1=2&c2=60...,https://www.webmd.com/search/search_results/de...,sb.scorecardresearch.com,UID=16F18428a36a38a2808d1cc1606506090; UIDR=16...,webmd,True
72,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""mb.moatads.com""],[""User-Agent"",""Mozi...",https://www.webmd.com/search/search_results/de...,https://mb.moatads.com/yi.js?ud=1&qn=%604%7BZE...,https://www.webmd.com/search/search_results/de...,mb.moatads.com,,webmd,True
75,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/search/search_results/de...,https://contextual.media.net/mcx.js?&rt=2&call...,https://www.webmd.com/search/search_results/de...,contextual.media.net,visitor-id=2495076903613114000V10,webmd,True
81,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/search/search_results/de...,https://contextual.media.net/rtbsmpubs.php?&gd...,https://www.webmd.com/search/search_results/de...,contextual.media.net,visitor-id=2495076903613114000V10,webmd,True
82,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/search/search_results/de...,https://contextual.media.net/rtbsmpubs.php?&gd...,https://www.webmd.com/search/search_results/de...,contextual.media.net,visitor-id=2495076903613114000V10,webmd,True
83,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/search/search_results/de...,https://contextual.media.net/rtbsmpubs.php?&gd...,https://www.webmd.com/search/search_results/de...,contextual.media.net,visitor-id=2495076903613114000V10,webmd,True
91,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""px.moatads.com""],[""User-Agent"",""Mozi...",https://www.webmd.com/search/search_results/de...,https://px.moatads.com/pixel.gif?e=17&i=WEBMD_...,https://www.webmd.com/search/search_results/de...,px.moatads.com,,webmd,True
